# **Setup & Data Preparation**

In [ ]:
import gdown
import os
import zipfile
import pandas as pd

drive_url = "https://drive.google.com/uc?id=1MBtxjgIjSickaraV4GKOkHEM9jfHwWcu"
zip_path = "/content/archive.zip"
extract_path = "/content/dataset"

gdown.download(drive_url, zip_path, quiet=False)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset downloaded and extracted!")

xlsx_path = os.path.join(extract_path, "ardata set.xlsx")
df = pd.read_excel(xlsx_path)

monument_descriptions = dict(zip(df["exhibits"], df["Text in English"]))

print("✅ Descriptions loaded!")


Downloading...
From (original): https://drive.google.com/uc?id=1MBtxjgIjSickaraV4GKOkHEM9jfHwWcu
From (redirected): https://drive.google.com/uc?id=1MBtxjgIjSickaraV4GKOkHEM9jfHwWcu&confirm=t&uuid=11ec5074-826d-4527-bfff-0c087d1c2a87
To: /content/archive.zip
100%|██████████| 5.29G/5.29G [00:56<00:00, 93.2MB/s]


✅ Dataset downloaded and extracted!
✅ Descriptions loaded!


# **Data Preprocessing**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow_from_directory(
    os.path.join(extract_path, "training_set"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = datagen.flow_from_directory(
    os.path.join(extract_path, "training_set"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

class_names = list(train_generator.class_indices.keys())

print("✅ Data Preprocessing Done!")


Found 3441 images belonging to 72 classes.
Found 843 images belonging to 72 classes.
✅ Data Preprocessing Done!


# **Load Pretrained Model & Modify for Training**

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dense(len(class_names), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

print("✅ Model Ready for Training!")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ Model Ready for Training!


# **Train Model & Save Every 5 Epochs**

In [ ]:
import time
from google.colab import drive
from tensorflow.keras.callbacks import ModelCheckpoint

drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/trained_model"

checkpoint_callback = ModelCheckpoint(
    filepath=f"{save_path}_epoch{{epoch:02d}}.h5",
    save_freq=5 * len(train_generator),
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)


print("✅ Google Drive mounted and checkpoint callback ready!")


Mounted at /content/drive
✅ Google Drive mounted and checkpoint callback ready!


# **Train the Model with Checkpoints**

In [ ]:
import os

def count_images(folder_path):
    total = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff')):
                total += 1
    return total

train_dir = '/content/drive/MyDrive/archive/training_set'
test_dir = '/content/drive/MyDrive/archive/test_set'

train_count = count_images(train_dir)
test_count = count_images(test_dir)

print(f"Training sample: {train_count}")
print(f"Test samples: {test_count}")


Training sample: 9822
Test samples: 511


In [ ]:

EPOCHS = 20


model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpoint_callback]
)

print("✅ Training Completed!")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 521s 5s/step - accuracy: 0.1583 - loss: 3.7530 - val_accuracy: 0.3535 - val_loss: 3.1413
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 494s 5s/step - accuracy: 0.4895 - loss: 2.1775 - val_accuracy: 0.5018 - val_loss: 2.1771
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 476s 4s/step - accuracy: 0.6435 - loss: 1.4739 - val_accuracy: 0.5409 - val_loss: 1.8687
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 487s 5s/step - accuracy: 0.7339 - loss: 1.0577 - val_accuracy: 0.6204 - val_loss: 1.5605
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7791 - loss: 0.8818

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:202: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 502s 5s/step - accuracy: 0.7794 - loss: 0.8809 - val_accuracy: 0.6928 - val_loss: 1.1618
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 481s 4s/step - accuracy: 0.8514 - loss: 0.5980 - val_accuracy: 0.7746 - val_loss: 0.8858
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 478s 4s/step - accuracy: 0.8957 - loss: 0.4322 - val_accuracy: 0.8031 - val_loss: 0.6950
Epoch 8/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 472s 4s/step - accuracy: 0.9230 - loss: 0.3046 - val_accuracy: 0.7936 - val_loss: 0.7260
Epoch 9/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 496s 5s/step - accuracy: 0.9275 - loss: 0.2607 - val_accuracy: 0.8019 - val_loss: 0.7096
Epoch 10/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 494s 5s/step - accuracy: 0.9468 - loss: 0.1982 - val_accuracy: 0.8458 - val_loss: 0.5309
Epoch 11/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 483s 4s/step - accuracy: 0.9582 - loss: 0.1592 - val_accuracy: 0.8826 - val_loss: 0.4749
Epoch 12/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 484s 4s/step - accuracy: 0.9587 - loss: 0.1408 - val_accuracy: 0.

# **Save The fine tuned model**

In [ ]:
import os
import json
from google.colab import drive

drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/trained_model"

model.save(f"{save_path}.h5")
print("✅ Model saved successfully!")

class_map_path = f"{save_path}_classes.json"
with open(class_map_path, "w") as f:
    json.dump(class_names, f)
print("✅ Class names saved!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved successfully!
✅ Class names saved!


# **Loading the model**

In [ ]:
import tensorflow as tf
import pandas as pd
import gdown

model_url = "https://drive.google.com/uc?id=1aO1YqsqEryhAFt61RnZxLTV0ESQPMN1_"
json_url = "https://drive.google.com/uc?id=10VPVoewDB9ZhqxAOYpA1w8sW8UX8QZQH"
xlsx_url = "https://drive.google.com/uc?id=1He3KtMi4R4X2Sjs66YaMUIeXpHzL4aSr"

model_path = "/content/monument_model.h5"
json_path = "/content/trained_model.json"
xlsx_path = "/content/monuments.xlsx"

gdown.download(model_url, model_path, quiet=False)
gdown.download(json_url, json_path, quiet=False)
gdown.download(xlsx_url, xlsx_path, quiet=False)

model = tf.keras.models.load_model(model_path)

df = pd.read_excel(xlsx_path)
monument_descriptions = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))

print("✅ Model loaded successfully!")
print("✅ Monument descriptions loaded successfully!")


# **Make Predictions & Retrieve Descriptions**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files
import json

class_names_path = "/content/trained_model.json"
with open(class_names_path, "r") as f:
    class_names = json.load(f)

print("✅ Class Names Loaded:", class_names)
IMG_SIZE = (224, 224)

def predict_monument(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions)
    predicted_class = class_names[predicted_index]

    description = monument_descriptions.get(predicted_class, "No description available.")

    return predicted_class, description

print("📤 Please upload an image for prediction:")
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
print(f"📸 Image received: {image_path}")

monument, desc = predict_monument(image_path)

print(f"🔹 Predicted Monument: {monument}")
print(f"📜 Information: {desc}")


# **Load the Model**

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("monument_model.h5")


# **Convert the Model to TFLite**

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model successfully converted to TensorFlow Lite!")


# **Optimize the TFLite Model**

In [ ]:

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

with open("model_quantized.tflite", "wb") as f:
    f.write(tflite_quantized_model)

print("✅ Quantized TensorFlow Lite model saved!")


# **Upload the Converted Model to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mv model.tflite /content/drive/MyDrive/
print("✅ Model uploaded to Google Drive!")